In [2]:
import urllib.request
import json
import pandas as pd
from pandas.io.json import json_normalize
from datetime import date, timedelta

from dateutil.relativedelta import relativedelta

In [3]:
key = '4263575945776f6c3130356a6d556b53'
target_date = 202111
url = 'http://openapi.seoul.go.kr:8088/%s/json/CardSubwayTime/1/5/%s/'%(key, target_date )

In [4]:
result = urllib.request.urlopen(url)
json_raw_data = result.read()
json_utf8 = json_raw_data.decode('utf-8')
json_complete = json.loads(json_utf8)
a = pd.DataFrame(json_complete['CardSubwayTime']['row'])

In [5]:
columns_name =a.columns
start_date = date(2019, 1, 1)
key = '4263575945776f6c3130356a6d556b53'

final_data = pd.DataFrame(columns = columns_name )
start_date = date(2019, 1, 1)
for i in range(1, 25):
    target_date = start_date.strftime('%Y%m')
    url = 'http://openapi.seoul.go.kr:8088/%s/json/CardSubwayTime/1/1000/%s/'%(key, target_date )
    result = urllib.request.urlopen(url)
    json_raw_data = result.read()
    json_utf8 = json_raw_data.decode('utf-8')
    json_complete = json.loads(json_utf8)
    monthly_data = pd.DataFrame(json_complete['CardSubwayTime']['row'])
    final_data= pd.concat([final_data, monthly_data])
    plus_month = relativedelta(months=1)
    start_date = start_date + plus_month
    
final_data

,USE_MON,LINE_NUM,SUB_STA_NM,FOUR_RIDE_NUM,FOUR_ALIGHT_NUM,FIVE_RIDE_NUM,FIVE_ALIGHT_NUM,SIX_RIDE_NUM,SIX_ALIGHT_NUM,SEVEN_RIDE_NUM,...,TWENTY_THREE_ALIGHT_NUM,MIDNIGHT_RIDE_NUM,MIDNIGHT_ALIGHT_NUM,ONE_RIDE_NUM,ONE_ALIGHT_NUM,TWO_RIDE_NUM,TWO_ALIGHT_NUM,THREE_RIDE_NUM,THREE_ALIGHT_NUM,WORK_DT
0,201901,경원선,옥수,0.0,0.0,0.0,7.0,19.0,21.0,34.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20190203
1,201901,1호선,서울역,927.0,22.0,10682.0,8774.0,13504.0,45545.0,39110.0,...,19442.0,3612.0,7167.0,0.0,410.0,0.0,0.0,0.0,0.0,20190203
2,201901,1호선,시청,74.0,0.0,2409.0,4049.0,3532.0,19241.0,7703.0,...,4079.0,981.0,1256.0,3.0,29.0,0.0,0.0,0.0,0.0,20190203
3,201901,1호선,종각,100.0,8.0,4409.0,5105.0,4593.0,26969.0,6733.0,...,5560.0,4886.0,1722.0,0.0,0.0,0.0,0.0,0.0,0.0,20190203
4,201901,1호선,종로3가,122.0,4.0,5061.0,2520.0,3876.0,12832.0,6066.0,...,5503.0,3214.0,2471.0,0.0,52.0,0.0,0.0,0.0,0.0,20190203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,202012,중앙선,오빈,0.0,0.0,99.0,1.0,356.0,230.0,690.0,...,65.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,20210103
599,202012,중앙선,양평,10.0,0.0,900.0,29.0,2116.0,1262.0,4918.0,...,1093.0,14.0,179.0,0.0,0.0,0.0,0.0,0.0,0.0,20210103
600,202012,중앙선,원덕,12.0,0.0,276.0,2.0,305.0,58.0,483.0,...,93.0,3.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,20210103
601,202012,중앙선,용문,215.0,1.0,553.0,10.0,1347.0,254.0,2058.0,...,640.0,0.0,130.0,0.0,0.0,0.0,0.0,0.0,0.0,20210103


In [6]:
final_data.drop(columns = ['WORK_DT'],inplace = True)

In [7]:
ride2019 = final_data[:7171]
ride2019 = ride2019.groupby(['LINE_NUM']) [list(final_data.loc[:,'FOUR_RIDE_NUM'::2].columns)].sum()
ride2019 = pd.DataFrame(ride2019.sum(axis=1),columns = ['RIDE'])
ride2019.sort_values(by = 'RIDE', axis=0, ascending=True, inplace = True)

In [8]:
ride2020 = final_data[7171:]
ride2020 = ride2020.groupby(['LINE_NUM']) [list(final_data.loc[:,'FOUR_RIDE_NUM'::2].columns)].sum()
ride2020 = pd.DataFrame(ride2020.sum(axis=1),columns = ['RIDE'])
ride2020.sort_values(by = 'RIDE', axis=0, ascending=True, inplace = True)

In [9]:
ride2019

,RIDE
LINE_NUM,
장항선,5718641.0
경강선,10330834.0
경춘선,14525071.0
우이신설선,16163995.0
수인선,16806916.0
9호선2~3단계,30584049.0
중앙선,36368017.0
경의선,39448985.0
과천선,40860939.0


In [10]:
ride2020 

,RIDE
LINE_NUM,
장항선,3503168.0
경강선,7984289.0
경춘선,10875185.0
우이신설선,12549800.0
수인선,13131225.0
9호선2~3단계,25032548.0
중앙선,26686506.0
공항철도 1호선,27360161.0
과천선,27908510.0
